In [2]:
%pip install -r /home/harsh/Documents/RAG/requirements -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

os.environ.get('OPENAI_API_KEY')

'sk-proj-WSb39FTU97JTHmcZg9oDT3BlbkFJ3qkIBAqdpbcnjuZwtV1S'

In [2]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model = 'gpt-3.5-turbo')
# output = llm.invoke("Explain quantum mechanics in one sentence")
# print(output.content)

In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage, 
    AIMessage
)

messages = [
    SystemMessage(content="You are a scientist and you respond only in Hindi"),
    HumanMessage(content = "Explain what is RAG in AI")
]
output = llm.invoke(messages)
print(output.content)

RAG (Retrieval-Augmented Generation) AI technique hai jo natural language processing (NLP) mein istemal hota hai. Is technique mein retrieval-based model aur generation-based model ko combine kiya jata hai. Retrieval-based model text ko samajhne aur relevant information ko retrieve karne mein mahir hota hai, jabki generation-based model naye text ko generate karne mein upyog hota hai. RAG AI technique ke dwara text generation ko improve kiya jata hai, kyunki yeh retrieval ki madad se context aur information ko better understand karta hai.


## Caching LLM responses

#### InMemoryCache

In [3]:
from langchain.globals import set_llm_cache # type: ignore
from langchain_openai import OpenAI    # type: ignore
llm = OpenAI(model_name = 'gpt-3.5-turbo-instruct')

In [19]:
%%time 
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 40.9 ms, sys: 2.16 ms, total: 43.1 ms
Wall time: 2.27 s


'\n\nWhy was the math book sad? Because it had too many problems!'

In [20]:
%%time
llm.invoke(prompt)

CPU times: user 737 μs, sys: 73 μs, total: 810 μs
Wall time: 823 μs


'\n\nWhy was the math book sad? Because it had too many problems!'

#### printing streamline

In [8]:
prompt = 'Tell me 5 joke that a toddler can understand'
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

1. Why did the banana go to the doctor? Because it wasn't peeling well!
2. Why did the cookie go to the doctor? Because it was feeling crumbly!
3. Why did the scarecrow win an award? Because he was outstanding in his field!
4. Why was the math book sad? Because it had too many problems!
5. Why did the tomato turn red? Because it saw the salad dressing!

#### Langchain prompting

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experience virologist
write a few sentences about {virus} in {language}
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(virus='corona', language = 'English')

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

The coronavirus, also known as COVID-19, is a highly contagious respiratory virus that has caused a global pandemic. It originated in Wuhan, China in late 2019 and has since spread to nearly every country in the world. The virus primarily spreads through respiratory droplets and can cause symptoms ranging from mild cold-like symptoms to severe respiratory illness and death. Efforts to control the spread of the virus include social distancing, wearing masks, and vaccination campaigns.


#### ChatPromptTemplates

In [15]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in JSON format'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population. Just give the ranking of the coutries and nothing else')
    ]
)

messages = chat_template.format_messages(n='10', area='Europe')

llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "1": "Germany",
    "2": "Russia",
    "3": "Turkey",
    "4": "France",
    "5": "United Kingdom",
    "6": "Italy",
    "7": "Spain",
    "8": "Ukraine",
    "9": "Poland",
    "10": "Romania"
}


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke({
    "name": "Bob",
    "user_input": "Tell me something about OpenAI company"
})

llm.invoke(prompt_value).content

'OpenAI is an artificial intelligence research lab that aims to ensure that artificial general intelligence (AGI) benefits all of humanity. They conduct research in various AI-related fields and work on developing cutting-edge AI technologies. Additionally, OpenAI promotes openness and collaboration in the field of AI research.'

#### Simple Chains

In [11]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist
write a few sentences about {virus} in {language}
'''

prompt_template = PromptTemplate.from_template(template = template)

chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

output = chain.invoke({'virus': 'corona', 'language': 'English'})



> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist
write a few sentences about corona in English


> Finished chain.


In [12]:
output

{'virus': 'corona',
 'language': 'English',
 'text': 'Coronavirus, also known as COVID-19, is a novel virus that emerged in late 2019 and has since spread rapidly across the globe, causing a pandemic. It is a highly contagious virus that primarily spreads through respiratory droplets when an infected person coughs or sneezes. Symptoms of COVID-19 can range from mild flu-like symptoms to severe respiratory illness, and can be particularly dangerous for the elderly and those with underlying health conditions. Efforts to control the spread of the virus have included widespread testing, social distancing, mask-wearing, and the development of vaccines. As a virologist, my expertise lies in studying the biology and behavior of viruses like coronavirus in order to better understand and combat them.'}

In [22]:
template = 'What is the capital of {country}? List the top 3 places to visit in that city. Use bullet points to show the answer'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

country = input('Enter country: ')
output = chain.invoke(country)



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Indi? List the top 3 places to visit in that city. Use bullet points to show the answer

> Finished chain.


In [23]:
output['text']

'The capital of India is New Delhi. \n\nSome of the top places to visit in New Delhi are:\n- Red Fort\n- India Gate\n- Qutub Minar'

#### Sequential Chaining

In [6]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.8)
prompt_template = PromptTemplate.from_template(
    template = "You are en experienced scientist and Python Programmer. Write a function that implements the concept of {function}"
)
chain1 = LLMChain(
    llm = llm1,
    prompt = prompt_template
)


llm2 = ChatOpenAI(model = 'gpt-4-turbo', temperature = 1.2)
prompt_template = PromptTemplate(
    template = "Given the python function {function}, describe it in detail"
)
chain2 = LLMChain(
    llm = llm2,
    prompt = prompt_template
)

final_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose = True)
output = final_chain.invoke('open-cv')
output



> Entering new SimpleSequentialChain chain...
Here is a simple function using the OpenCV library in Python to read an image file and display it:

```python
import cv2

def display_image(image_path):
    # Read the image file
    image = cv2.imread(image_path)
    
    # Display the image
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = 'example.jpg'
display_image(image_path)
```

This function reads an image file from the specified path using `cv2.imread()` and displays it using `cv2.imshow()`. The function `cv2.waitKey(0)` waits indefinitely for a key press to close the image window, and `cv2.destroyAllWindows()` closes all OpenCV windows.

The function `display_image` in the provided Python script is designed to load and show an image using the OpenCV library, which is a popular library for computer vision tasks. The script is structured into three main parts: the import statement, the function definition, and an example us

{'input': 'open-cv',
 'output': 'The function `display_image` in the provided Python script is designed to load and show an image using the OpenCV library, which is a popular library for computer vision tasks. The script is structured into three main parts: the import statement, the function definition, and an example usage. Here are the details of each part:\n\n1. **Import Statement:**\n   ```python\n   import cv2\n   ```\n   This line imports the OpenCV library and binds it to the name `cv2`. This import is necessary to access functions from the OpenCV library, which are used to read, display, and manipulate the image.\n\n2. **Function Definition:**\n   ```python\n   def display_image(image_path):\n       # Read the image file\n       image = cv2.imread(image_path)\n       \n       # Display the image\n       cv2.imshow(\'Image\', image)\n       cv2.waitKey(0)\n       cv2.destroyAllWindows()\n   ```\n   - **Function Signature (`display_image(image_path)`):**\n     The function is nam

In [10]:
print(output['output'])

The function `display_image` in the provided Python script is designed to load and show an image using the OpenCV library, which is a popular library for computer vision tasks. The script is structured into three main parts: the import statement, the function definition, and an example usage. Here are the details of each part:

1. **Import Statement:**
   ```python
   import cv2
   ```
   This line imports the OpenCV library and binds it to the name `cv2`. This import is necessary to access functions from the OpenCV library, which are used to read, display, and manipulate the image.

2. **Function Definition:**
   ```python
   def display_image(image_path):
       # Read the image file
       image = cv2.imread(image_path)
       
       # Display the image
       cv2.imshow('Image', image)
       cv2.waitKey(0)
       cv2.destroyAllWindows()
   ```
   - **Function Signature (`display_image(image_path)`):**
     The function is named `display_image` and takes one parameter, `image_path

### Langchain agents

In [11]:
%pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_experimental.utilities import PythonREPL #type: ignore

python_repl = PythonREPL()
python_repl.run("print([n for n in range(1, 100) if n%13 == 0])")

'[13, 26, 39, 52, 65, 78, 91]\n'

In [12]:
from langchain_experimental.agents.agent_toolkits import create_python_agent     # type: ignore
from langchain_experimental.tools.python.tool import PythonREPLTool              # type: ignore

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4-turbo-preview', temperature = 0)
agent_executor = create_python_agent(
    llm = llm,
    tool = PythonREPLTool(),
    verbose = True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math` module's `factorial` function. Then, I will calculate the square root of that result using the `sqrt` function from the same module. Finally, I will format the result to display it with 4 decimal points using the `format` function.

Action: Python_REPL
Action Input: import math
Observation: 
Thought:Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Observation: 
Thought:Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
Observation: 
Thought:Action: Python_REPL
Action Input: print(format(sqrt_factorial_12, '.4f'))
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [16]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3 ?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [21]:
response

{'input': 'What is the answer to 5.1 ** 7.3 ?', 'output': '146306.05007233328'}

### Langchain tools: DuckDuckGo and Wikipedia

In [22]:
%pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.run('Freddie Mercury and Queen')
output

'Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock\'s most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury always talked about his pride at writing "Killer Queen," which appeared on Queen\'s 1974 album Sheer Heart Attack. He said it was written in one night and was a song that he ... Outside of his work with Queen, Freddie Mercury recorded some solo material, including two solo albums and several singles. Read more: Queen\'s 20 greatest ever songs His two solo albums were Mr. Bad Guy (1985) and Barcelona (1988). In 1993, a remix of \'Living on My Own\', posthumously reached number one in the UK. Queen are a British rock band formed in London in 1970 by Freddie Mercury (lead vocals, piano

In [7]:
search.name, search.description

('duckduckgo_search',
 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [10]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
wrapper = DuckDuckGoSearchAPIWrapper(region = 'in-en', max_results=3, safesearch='moderate')
output = wrapper.run('New delhi')

In [12]:
print(output)

New Delhi is the national capital of India, founded in 1912 as the new seat of British colonial rule. It has a modern and spacious layout, contrasting with the ancient and crowded Old Delhi, and is home to many government buildings, museums, and parks. Delhi is the capital of India and a World Heritage site, with a rich and diverse history and culture. Learn about its geography, landmarks, wildlife, and climate from Britannica's experts. Discover the best places to visit in Delhi, India, a city with a rich history and diverse culture. Explore its ancient monuments, gardens, museums, temples, and markets in both New Delhi and Old Delhi.


In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llama-index'})

'Page: Vector database\nSummary: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corr'

### Creating a ReAct agent

In [7]:
%pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [13]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)

True

In [6]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool

llm = ChatOpenAI(model = 'gpt-3.5-turbo')

template = '''
Answer the following questions as best you can.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
print(type(prompt))
print(prompt.input_variables)
print(prompt.template)

LangSmithUserError: API key must be provided when using hosted LangSmith API